In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from google.colab import files
import io
uploaded = files.upload()
data = pd.read_csv(io.BytesIO(uploaded['zomato.csv']))
data = data[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Aggregate rating']].dropna()
data = data.rename(columns={
    'Restaurant Name': 'name',
    'Cuisines': 'category',
})
data['description'] = data['category'].astype(str)
user_preferences = {
    'preferred_categories': ['North Indian', 'Chinese', 'Italian'],
    'liked_keywords': ['spicy', 'pizza', 'noodles', 'butter chicken']
}
sentiment_model = pipeline("sentiment-analysis")
user_input_mood = "I'm feeling really happy and excited today!"
mood_result = sentiment_model(user_input_mood)[0]
mood_score = 1 if mood_result['label'] == 'POSITIVE' else 0.5
vectorizer = TfidfVectorizer()
desc_matrix = vectorizer.fit_transform(data['description'])
query = ' '.join(user_preferences['liked_keywords'])
query_vector = vectorizer.transform([query])
similarities = cosine_similarity(query_vector, desc_matrix).flatten()
data['similarity'] = similarities
filtered = data[data['category'].str.contains('|'.join(user_preferences['preferred_categories']), case=False, na=False)].copy()
filtered['final_score'] = filtered['similarity'] * mood_score
recommendations = filtered.sort_values(by='final_score', ascending=False)
print("\n🎯 Top Personalized Recommendations:")
print(recommendations[['name', 'category', 'final_score']].head(10))


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Saving zomato.csv to zomato.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu



🎯 Top Personalized Recommendations:
                       name             category  final_score
9237  Pizzeria Vaatika Cafe       Pizza, Chinese     0.881355
256            Fong's Pizza       Chinese, Pizza     0.881355
8442        Hurry's Paratha  Pizza, North Indian     0.849603
3069                 Sbarro       Pizza, Italian     0.765897
201                Deorio's       Italian, Pizza     0.765897
8415                 Sbarro       Pizza, Italian     0.765897
1507           Crust Bistro       Italian, Pizza     0.765897
228   Pie Slingers Pizzeria       Italian, Pizza     0.765897
2623    Amici Gourmet Pizza       Pizza, Italian     0.765897
8277     Pizza Hut Delivery       Italian, Pizza     0.765897
